In [1]:
import pandas as pd

In [2]:
!ls

drd2
get_HierVAE_score_and_validity.ipynb
logp04
logp06
qed
run_hie_val_drd2-2021.04.22-23.31.02.log
run_hie_val_logp04-2021.04.22-23.35.49.log
run_hie_val_qed-2021.04.22-23.30.26.log
val_results_2021.04.22-23.30.27
val_results_2021.04.22-23.31.03
val_results_2021.04.22-23.35.50
val_results_2021.04.23-12.52.33


In [3]:
import glob, os
import numpy as np

In [4]:
val_results_dirs = glob.glob('val_results_2021.04.*')

In [51]:
!pwd

/Users/brian/Downloads/hiervae


In [49]:
os.chdir('/Users/brian/Downloads/hiervae/')

In [50]:
cwd = os.getcwd()

In [73]:
import rdkit
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors

In [77]:
def similarity(a, b):
    if a is None or b is None:
        return 0.0
    amol = Chem.MolFromSmiles(a)
    bmol = Chem.MolFromSmiles(b)
    if amol is None or bmol is None:
        return 0.0

    fp1 = AllChem.GetMorganFingerprintAsBitVect(amol, 2, nBits=2048, useChirality=False)
    fp2 = AllChem.GetMorganFingerprintAsBitVect(bmol, 2, nBits=2048, useChirality=False)
    return DataStructs.TanimotoSimilarity(fp1, fp2)

def get_logp_results(paths, num_decode=20, delta=0.4):    
    cwd = os.getcwd()

    for d in paths:
        os.chdir(d)
        results_files = glob.glob('results.*')
        for f in results_files:
            df = pd.read_csv(f,sep='\s+',header=None)
            data = df.values.tolist()
            
            data = [(a,b,float(c),float(d)) for a,b,c,d in data]
            n_mols = len(data) / num_decode
            print(f, len(data), num_decode)
#             assert len(data) % num_decode == 0

            all_logp = []

            for i in range(0, len(data), num_decode):
                set_x = set([x[0] for x in data[i:i+num_decode]])
                assert len(set_x) == 1

                good = [(sim,logp) for _,_,sim,logp in data[i:i+num_decode] if 1 > sim >= delta]
                if len(good) > 0:
                    sim,logp = max(good, key=lambda x:x[1])
                    all_logp.append(max(0,logp))
                else:
                    all_logp.append(0.0) #No improvement

            assert len(all_logp) == n_mols
            all_logp = np.array(all_logp)

            print('Evaluated on %d samples' % (n_mols,))
            print('average improvement', np.mean(all_logp), 'stdev',  np.std(all_logp))
    os.chdir(cwd)
            
            
def get_drd2_results(paths, num_decode=20, sim_delta=0.4, prop_delta=0.5):    
    cwd = os.getcwd()
    for d in paths:
        os.chdir(d)
        results_files = glob.glob('results.*')
        for f in results_files:
            df = pd.read_csv(f,sep='\s+',header=None)
            data = df.values.tolist()
            
            data = [(a,b,float(c),float(d)) for a,b,c,d in data]
            n_mols = len(data) / num_decode

            n_succ = 0.0
            for i in range(0, len(data), num_decode):
                set_x = set([x[0] for x in data[i:i+num_decode]])
                assert len(set_x) == 1

                good = [(sim,prop) for _,_,sim,prop in data[i:i+num_decode] if 1 > sim >= sim_delta and prop >= prop_delta]
                if len(good) > 0:
                    n_succ += 1            
            print('Evaluated on %d samples' % (n_mols,))
            print('success rate', n_succ / n_mols)    
    os.chdir(cwd)
    
def get_qed_results(paths, num_decode=20, sim_delta=0.4, prop_delta=0.9):

    num_decode=20
    sim_delta=0.4
    prop_delta=0.9
    cwd = os.getcwd()
    for d in paths:
        os.chdir(d)
        results_files = glob.glob('results.*')
        for f in results_files:
            df = pd.read_csv(f,sep='\s+',header=None)
            data = df.values.tolist()
            data = [(a,b,float(c),float(d)) for a,b,c,d in data]
            n_mols = len(data) / num_decode

            n_succ = 0.0
            for i in range(0, len(data), num_decode):
                set_x = set([x[0] for x in data[i:i+num_decode]])
                assert len(set_x) == 1

                good = [(sim,prop) for _,_,sim,prop in data[i:i+num_decode] if 1 > sim >= sim_delta and prop >= prop_delta]
                if len(good) > 0:
                    n_succ += 1

            print('Evaluated on %d samples' % (n_mols,))
            print('success rate', n_succ / n_mols)
    os.chdir(cwd)

    
def get_diversity(paths, num_decode=20, sim_delta=0.4, prop_delta=0.9):

    cwd = os.getcwd()
    def convert(x):
        return None if x == "None" else x

    for d in paths:
        os.chdir(d)
        results_files = glob.glob('results.*')
        for f in results_files:
            df = pd.read_csv(f,sep='\s+',header=None)
            data = df.values.tolist()
            data = [(a,b,float(c),float(d)) for a,b,c,d in data]

            all_div = []
            n_succ = 0
            for i in range(0, len(data), num_decode):
                set_x = set([x[0] for x in data[i:i+num_decode]])
                assert len(set_x) == 1

                good = [convert(y) for x,y,sim,prop in data[i:i+num_decode] if sim >= sim_delta and prop >= prop_delta]
                if len(good) == 0:
                    continue

                good = list(set(good))
                if len(good) == 1:
                    all_div.append(0.0)
                    continue
                n_succ += 1

                div = 0.0
                tot = 0
                for i in range(len(good)):
                    for j in range(i + 1, len(good)):
                        div += 1 - similarity(good[i], good[j])
                        tot += 1
                div /= tot
                all_div.append(div)

            all_div = np.array(all_div)
            print(np.mean(all_div), np.std(all_div))
            print(n_succ)
    os.chdir(cwd)



## logP04 scores

In [68]:
get_logp_results([os.path.join(cwd,'val_results_2021.04.22-23.35.50','logp04')])

results.2 4000 20
Evaluated on 200 samples
average improvement 3.0341900748431248 stdev 1.8566762763377564
results.16 4000 20
Evaluated on 200 samples
average improvement 2.8557835290099423 stdev 1.818905510617248
results.5 4000 20
Evaluated on 200 samples
average improvement 3.189793042364847 stdev 1.8198943283268407
results.11 4000 20
Evaluated on 200 samples
average improvement 3.19010021865025 stdev 1.7516875581760853
results.18 4000 20
Evaluated on 200 samples
average improvement 2.9465320665723778 stdev 1.784627076280205
results.19 4000 20
Evaluated on 200 samples
average improvement 2.988526164066667 stdev 1.8125143672352202
results.4 4000 20
Evaluated on 200 samples
average improvement 3.2151282004822526 stdev 1.7195997744858627
results.10 4000 20
Evaluated on 200 samples
average improvement 3.246232111609067 stdev 1.8029130864448852
results.3 4000 20
Evaluated on 200 samples
average improvement 3.1204526798102745 stdev 1.7796681270138055
results.17 4000 20
Evaluated on 200 sam

In [83]:
get_diversity([os.path.join(cwd,'val_results_2021.04.22-23.35.50','logp04')],num_decode=20, sim_delta=0.4, prop_delta=0.0)

0.3994968337074946 0.2753039632493079
125
0.38920432985723924 0.3035283385094287
109
0.4431373908826988 0.2613764852986803
137
0.44949770446242404 0.2691518190950568
137
0.4029500704926486 0.3027524748633366
112
0.3821952172015352 0.2958360917331136
111
0.4785131578661714 0.25302108084157193
145
0.45044710360279455 0.2715241212997659
135
0.42225578922519796 0.27880548288895696
129
0.38997859944407687 0.30033142766022825
108
0.4269919582214241 0.27640053231032
135
0.4235883848828759 0.286595685937378
126
0.40881394977037644 0.281229601069048
115
0.40279802006768284 0.29907285999834876
111
0.444100120534312 0.26887420888224844
140
0.4317332347232526 0.2862429136495269
61
0.2955630036404704 0.27174910390763957
84
0.3995345388257621 0.30039160322309433
112
0.42531212260362644 0.27007441104452795
136
0.45241760189167396 0.292344988168215
127


## logp06 improvement scores

In [69]:
get_logp_results([os.path.join(cwd,'val_results_2021.04.23-12.52.33','logp06')],0.6)

results.2 4000 20
Evaluated on 200 samples
average improvement 1.900635407689014 stdev 1.2950571456475979
results.16 4000 20
Evaluated on 200 samples
average improvement 2.215952509780583 stdev 0.9965525711045247
results.5 4000 20
Evaluated on 200 samples
average improvement 2.159993042768345 stdev 1.243975480988262
results.11 4000 20
Evaluated on 200 samples
average improvement 2.192220716827863 stdev 1.0364181069636031
results.18 4000 20
Evaluated on 200 samples
average improvement 2.060774355853315 stdev 0.9396289445242986
results.19 4000 20
Evaluated on 200 samples
average improvement 2.153642591290125 stdev 0.9401271566294314
results.4 4000 20
Evaluated on 200 samples
average improvement 2.0677409530973567 stdev 1.142641586831391
results.10 4000 20
Evaluated on 200 samples
average improvement 2.209495323313261 stdev 0.9891011572683833
results.3 4000 20
Evaluated on 200 samples
average improvement 2.003674488433429 stdev 1.152626738666996
results.17 4000 20
Evaluated on 200 samples

In [81]:
get_diversity([os.path.join(cwd,'val_results_2021.04.23-12.52.33','logp06')],num_decode=20, sim_delta=0.6, prop_delta=0.0)

0.2885990559106227 0.18845231771578858
140
0.32498852685499224 0.18009010268598388
156
0.3136346506484048 0.17550686042293417
154
0.33046770500388556 0.1712678235366104
159
0.33833860424102796 0.1828127517051943
152
0.35007955021339354 0.17606196291088216
155
0.2890793400075756 0.17857821335429208
142
0.3463850832822717 0.15961921146016508
169
0.2952276638806024 0.18189478306752624
142
0.3201722572952278 0.19299866998104073
149
0.32853635828017186 0.17125522200636759
159
0.33524664309888463 0.1766438482151403
154
0.2816604882383559 0.1979924582402867
125
0.34519893160423537 0.17601605617969537
160
0.3450634896646098 0.17719038197745005
158
0.3127096355254459 0.18381007890012868
148
0.200640214933131 0.19763646141603458
78
0.3345878325098431 0.17640143886830126
156
0.3313515590432003 0.1811032387134447
156
0.3433022644855796 0.17627110787583417
157


## QED improvement scores

In [70]:
get_qed_results([os.path.join(cwd,'val_results_2021.04.22-23.30.27','qed')])

Evaluated on 360 samples
success rate 0.6166666666666667
Evaluated on 360 samples
success rate 0.5888888888888889
Evaluated on 360 samples
success rate 0.6277777777777778
Evaluated on 360 samples
success rate 0.55
Evaluated on 360 samples
success rate 0.5583333333333333
Evaluated on 360 samples
success rate 0.5833333333333334
Evaluated on 360 samples
success rate 0.6138888888888889
Evaluated on 360 samples
success rate 0.5944444444444444
Evaluated on 360 samples
success rate 0.6277777777777778
Evaluated on 360 samples
success rate 0.5861111111111111
Evaluated on 360 samples
success rate 0.6083333333333333
Evaluated on 360 samples
success rate 0.5555555555555556
Evaluated on 360 samples
success rate 0.6361111111111111
Evaluated on 360 samples
success rate 0.6055555555555555
Evaluated on 360 samples
success rate 0.6083333333333333
Evaluated on 360 samples
success rate 0.6
Evaluated on 360 samples
success rate 0.49444444444444446
Evaluated on 360 samples
success rate 0.5805555555555556
Ev

In [78]:
get_diversity([os.path.join(cwd,'val_results_2021.04.22-23.30.27','qed')],num_decode=20, sim_delta=0.4, prop_delta=0.5)

0.4244802559918663 0.27791983134853565
206
0.3743139019053508 0.3262719776228727
152
0.41947933507580554 0.2918131680438112
190
0.37143461975796854 0.3264061056079409
147
0.34708426860941327 0.33503179381850523
132
0.3459682925473561 0.3299166846311292
136
0.4309533106402332 0.2903747477390817
203
0.35213466118876735 0.3241024427177795
149
0.41167557758752404 0.2960520254049341
200
0.3662659223544736 0.33504559389896976
136
0.4032449998990812 0.30970371903145827
176
0.3485193429272803 0.3243475731265634
141
0.4438838990613861 0.2566668963897538
235
0.35907199913406973 0.32815444844382985
152
0.3979867171253874 0.314502796761012
176
0.37983026535351333 0.3308943450123259
156
0.3361785205835835 0.2750560493295374
160
0.35681583462975736 0.329129806962771
147
0.4002174733753605 0.30763916663429286
172
0.354711536621331 0.3325713495680257
144


## DRD2 scores

In [71]:
get_drd2_results([os.path.join(cwd,'val_results_2021.04.22-23.31.03','drd2')])

Evaluated on 500 samples
success rate 0.566
Evaluated on 500 samples
success rate 0.728
Evaluated on 500 samples
success rate 0.67
Evaluated on 500 samples
success rate 0.706
Evaluated on 500 samples
success rate 0.742
Evaluated on 500 samples
success rate 0.724
Evaluated on 500 samples
success rate 0.646
Evaluated on 500 samples
success rate 0.744
Evaluated on 500 samples
success rate 0.592
Evaluated on 500 samples
success rate 0.744
Evaluated on 500 samples
success rate 0.7
Evaluated on 500 samples
success rate 0.73
Evaluated on 500 samples
success rate 0.466
Evaluated on 500 samples
success rate 0.742
Evaluated on 500 samples
success rate 0.716
Evaluated on 500 samples
success rate 0.71
Evaluated on 500 samples
success rate 0.388
Evaluated on 500 samples
success rate 0.734
Evaluated on 500 samples
success rate 0.712
Evaluated on 500 samples
success rate 0.736


In [75]:
get_diversity([os.path.join(cwd,'val_results_2021.04.22-23.31.03','drd2')])

0.09093249961134006 0.17246488350368416
33
0.03825820786172062 0.1288001866472493
14
0.09110847145693605 0.18777453709273123
33
0.04885126553792322 0.14046243486760407
18
0.06506553396969098 0.164016110286474
23
0.07851810717923277 0.17672266561223057
29
0.07635828137054075 0.17704623048330542
24
0.05680214503885669 0.1474935892023224
23
0.0945650326540617 0.20356385812886088
25
0.05978732086111067 0.16143674847362638
20
0.05052691342035882 0.15566125930406885
17
0.07407201034717541 0.18325797905271865
23
0.07453316810658449 0.1598071795232366
20
0.06344531729470702 0.1567156145350329
23
0.0803950123344753 0.18311306594116725
29
0.07601593069070164 0.18376174229431846
27
0.04169209658614862 0.13390677899518558
11
0.08734087721741358 0.18426056264073312
29
0.0616429330623227 0.16651244777612287
23
0.059273954217969514 0.1480921887307037
23
